Data for sales/penetration will not be ready until after the 8th of the month.

### ***Pull Construction Funding at the beginning of the month, only once a month

October 2022 funding data - already pulled 10/3/22 - do not pull until after 11/1

In [1]:
import os
import sys
import pandas as pd
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

In [2]:
#crdmprd
conn = cx_Oracle.connect(
    user="AD21236",
    password="C3ntury123",
    dsn="crdm.qintra.com:1526/crdmprd",
    encoding="UTF-8")

# FRAM001P
conn2 = cx_Oracle.connect(
    user="AD21236",
    password="L2022$Um",
    dsn="RACORAP08-SCAN.CORP.INTRANET/FRAM001P_PROD",
    encoding="UTF-8")

#CCDWO1P - SALES
conn3 = cx_Oracle.connect(
    user="AD21236",
    password="C#ntury123",
    dsn="racorap33-scan.corp.intranet:1521/ccdw01p_users",
    encoding="UTF-8")

#DWPR01P
conn4 = cx_Oracle.connect(
    user="AD21236",
    password="C#ntury123",
    dsn="racorap33-scan.corp.intranet/dwpr01p_users",
    encoding="UTF-8")

In [6]:
%%time
#CBSA WCs

# query = """    #commented out to use separate wire center datasource

#             SELECT  STATE_CD AS STATE
#                     ,DMA_NM AS DMA
#                     ,CBSA_NM AS CBSA
#                     ,CENT_OFFC_CD AS PRIMARY_WIRE_CENTER_ID

#             FROM  usr.srv_metric_ctl_tier_t
#             WHERE   LGCY_CO_MGT_TIER_ID = 'LUMEN'
#                     AND CENT_OFFC_CD NOT LIKE '9%'
#                     AND STATE_CD <> 'XX'
#             ORDER BY   STATE_CD
#                        ,CENT_OFFC_CD 

# """

# LU_QUAL_O qualifications for XGS - update date here
query3 = """
            WITH QD AS (
            SELECT  
                    O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.LUID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    ,O.UNI_ECD
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND uni_ecd >= '1-OCT-2022'
                    AND uni_ecd < '1-NOV-2022'
                    
            )
            
            SELECT  PRIMARY_WIRE_CENTER_ID
                    ,to_char(UNI_ECD, 'Mon-YY') as build_date
                    ,count(distinct LUID) as lu_qual_cnt
                    
            FROM    QD
            
            GROUP BY
                    PRIMARY_WIRE_CENTER_ID
                    ,to_char(UNI_ECD, 'Mon-YY')

                                """

# LU_QUAL_O qualifications for GPON - update date here
query4 = """
            WITH QD AS (
            SELECT  
                    O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.LUID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    ,O.UNI_ECD
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND TECH_TYPE = 'GPON' 
                    AND uni_status='IN SERVICE'
                    AND uni_ecd >= '1-OCT-2022'
                    AND uni_ecd < '1-NOV-2022'
                    
            )
            
            SELECT  PRIMARY_WIRE_CENTER_ID
                    ,to_char(UNI_ECD, 'Mon-YY') as build_date
                    ,count(distinct LUID) as lu_qual_cnt
                    
            FROM    QD
            
            GROUP BY
                    PRIMARY_WIRE_CENTER_ID
                    ,to_char(UNI_ECD, 'Mon-YY')

                                """



# wc_ora_df = pd.read_sql(query, con=conn4) # commented out to use separate wire center data source
luqualo_ora_xgs_df = pd.read_sql(query3, con=conn2)
luqualo_ora_gpon_df = pd.read_sql(query4, con=conn2)

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 15.6 ms
Wall time: 1.05 s


In [8]:
%%time

#getting sales data separate because it takes a while to run - update date here

query4 = """

WITH HH AS (         --comes from corp strategy

    select count(t.ntwk_source_key) as lu_cnt
            ,t.snapshot_date
            ,t.WIRE_CENTER_ID
            ,t.CBSA_NAME
            ,t.STATE_CD
            ,CASE WHEN t.qualtech NOT IN ('XGSPON', 'GPON') THEN 'COPPER'
                  ELSE t.qualtech END AS qualtech  --In Tableau dashboard fiber = XGSPON,GPON/ everything else = copper 

    from 
            rpcarte.dd_t5846_hh_fp_sbg t
 
    where 
            t.snapshot_date = '30-SEP-2022'
            and t.SBG_MKT_LVL1_CUR = 'MASSMKT'
            and t.LU_COUNTABLE = 'Y'
            and t.HSI_MAX_QUAL <> 0 -- speed filter excluding 0 speed group
            and ILEC_OPS = 'LUMEN'
 
    group by 
            t.snapshot_date
            ,t.WIRE_CENTER_ID
            ,t.CBSA_NAME
            ,t.STATE_CD
            ,CASE WHEN t.qualtech NOT IN ('XGSPON', 'GPON') THEN 'COPPER'
                  ELSE t.qualtech END


), SALES AS (    --Ins: completed sale orders for hsi to xgspon enabled sddresses

    select  sum(t.units) as units
            ,t.transaction_date
            ,t.metric
            ,t.WIRE_CENTER_ID
            ,t.CBSA_NAME
            ,t.STATE_CD
            ,CASE WHEN t.BILLED_TECHNOLOGY = 'Copper' THEN 'COPPER'
                  ELSE t.qualtech END AS qualtech
       
    from    rpcarte.dd_t5846_units_sbg t
    where   t.transaction_date = '30-SEP-2022'
            and t.BILLED_TECHNOLOGY in ('Fiber', 'Copper') -- what technology the customer is actually being billed for
            and t.crnt_gl_cust_type_cd in ('R','S')
            and t.META_SRC_SYS_CD not in ('CTL_ENS_OVRLY') 
            and t.META_SRC_SYS_CD||metric not in ('Q_IR_CPROD_OVRLYEIS')
            and nvl(UPPER(t.PREPAID),'NA')||t.META_SRC_SYS_CD NOT IN ('INSTALINKPPP','INSTALINKQ_IR_CPROD','INSTALINKCTL_ENS')
            and t.ILEC_OPS = 'LUMEN'
            

        
    group by    t.transaction_date
                ,t.metric
                ,t.WIRE_CENTER_ID
                ,t.CBSA_NAME
                ,t.STATE_CD
                ,CASE WHEN t.BILLED_TECHNOLOGY = 'Copper' THEN 'COPPER'
                      ELSE t.qualtech END
                
), EIS AS (

    select  distinct 
            units
            ,transaction_date
            ,WIRE_CENTER_ID
            ,CBSA_NAME
            ,STATE_CD
            ,qualtech
       
    from    SALES
    where   metric = 'EIS'

)

SELECT  distinct
        to_char(h.snapshot_date, 'Mon-YY') as month
        ,h.WIRE_CENTER_ID
        ,h.CBSA_NAME
        ,h.STATE_CD
        ,h.qualtech
        ,h.lu_cnt as HH_cnt
        ,coalesce(e.units,0) as EIS
        ,coalesce((e.units / h.lu_cnt),0) as penetration_rate
        ,coalesce(s.units,0) as Ins_Sales

FROM    HH h
        LEFT JOIN EIS e
            on h.snapshot_date = e.transaction_date
            and h.WIRE_CENTER_ID = e.WIRE_CENTER_ID
            and h.CBSA_NAME = e.CBSA_NAME
            and h.STATE_CD = e.STATE_CD
            and h.qualtech = e.qualtech
        LEFT JOIN SALES s
            on h.snapshot_date = s.transaction_date
            and h.WIRE_CENTER_ID = s.WIRE_CENTER_ID
            and h.CBSA_NAME = s.CBSA_NAME
            and h.STATE_CD = s.STATE_CD
            and h.qualtech = s.qualtech
            and s.metric = 'Ins'

"""



# Best to run 1-3 months of data at a time

sales_ora_df = pd.read_sql(query4, con=conn3)

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 46.9 ms
Wall time: 8min 50s


In [9]:
# check sales data
sales_ora_df.head()

,MONTH,WIRE_CENTER_ID,CBSA_NAME,STATE_CD,QUALTECH,HH_CNT,EIS,PENETRATION_RATE,INS_SALES
0,Sep-22,PHNXAZNO,"PHOENIX-MESA-SCOTTSDALE, AZ",AZ,COPPER,39832,8482,0.212944,56
1,Sep-22,VRMLSDCO,"VERMILLION, SD",SD,COPPER,5867,846,0.144196,4
2,Sep-22,BLTNMNSO,"MINNEAPOLIS-ST. PAUL-BLOOMINGTON, MN-WI",MN,COPPER,19610,6200,0.316165,18
3,Sep-22,EVTNWYMA,"EVANSTON, WY",WY,COPPER,5641,1394,0.247119,10
4,Sep-22,SHLBMTMA,NON-CBSA,MT,COPPER,1658,956,0.576598,26


In [10]:
# Import wire center datasource from 'Get Accurate Wire Center Base' project
Sales_WC_excel = pd.read_excel(r'WireCenterBase/FINAL_wire_center_mapping_DO-NOT-ALTER.xlsx',sheet_name = 'Sheet1')
Sales_WC_excel.head()

,WIRE_CENTER_ID,STATE,CBSA_NAME,DMA_NAME
0,ABRDSDCO,SD,"ABERDEEN, SD",SIOUX FALLS (MITCHELL) SD
1,ABRDWA01,WA,"ABERDEEN, WA",SEATTLE-TACOMA WA
2,ACKLIACO,IA,NON-CBSA,DES MOINES-AMES IA
3,ADAROR21,OR,"CORVALLIS, OR",EUGENE OR
4,ADELIACO,IA,"DES MOINES-WEST DES MOINES, IA",DES MOINES-AMES IA


In [11]:
# Sales_WC_excel - contains cumulative wire center/cbsa/dma data
# luqualo_ora_xgs_df - contains enabled living units data for xgs, for selected month
# luqualo_ora_gpon_df - contains enabled living units data for gpon, for selected month
# sales_ora_df - contains sales data by wire center, for selected month

In [12]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

In [13]:
# clean base table
# check all states are legit
# Sales_WC_excel['STATE'].value_counts()

wc_df = Sales_WC_excel.copy() #save base table

In [14]:
wc_df = wc_df.rename(columns = {'WIRE_CENTER_ID': 'PRIMARY_WIRE_CENTER_ID',
                                'CBSA_NAME': 'CBSA',
                                'DMA_L2': 'DMA'})

In [15]:
# Import excel MG readiness data
MG_excel_df = pd.read_excel(r'081622_OpportunityScore_MultiGig_Readiness.xlsx',sheet_name = 'Datasheet')

# Import excel opportunity score data
OppScore_excel_df = pd.read_excel(r'OpportunityScoresMapping/CLLI_ZIP_XREF_WEIGHTED_SCORE.xlsx',sheet_name = 'Final')

In [16]:
# join in opportunity score
wc_ora_df = wc_df.merge(OppScore_excel_df[['CLLI','Weighted Opportunity Score']], how = 'left', left_on = 'PRIMARY_WIRE_CENTER_ID', right_on = 'CLLI')

In [17]:
wc_ora_df = wc_ora_df.rename(columns = {'DMA_NAME': 'DMA'})

In [18]:
# clean data
wc_ora_df = wc_ora_df[['STATE','DMA','CBSA','Weighted Opportunity Score','PRIMARY_WIRE_CENTER_ID']].rename(columns = {'Weighted Opportunity Score':'OPPORTUNITY_SCORE'})

In [19]:
# clean mg readiness data - getting core status and monthly planned build - month part needs to be hardcoded manually
MG_excel_df = MG_excel_df[['PRIMARY_WIRE_CENTER_ID','Core Status','Sep','Oct','Nov','Dec']].rename(columns = {'Core Status': 'CORE_READINESS',
                                                                                                              'Sep': 'SEP-22_PLANNED_XGS_LU',
                                                                                                              'Oct': 'OCT-22_PLANNED_XGS_LU',
                                                                                                              'Nov': 'NOV-22_PLANNED_XGS_LU',
                                                                                                              'Dec': 'DEC-22_PLANNED_XGS_LU'})

In [20]:
# join in planned xgs build/core status

wc_ora_df = wc_ora_df.merge(MG_excel_df, how = 'left', on = 'PRIMARY_WIRE_CENTER_ID')

In [21]:
# check data
wc_ora_df.head()

,STATE,DMA,CBSA,OPPORTUNITY_SCORE,PRIMARY_WIRE_CENTER_ID,CORE_READINESS,SEP-22_PLANNED_XGS_LU,OCT-22_PLANNED_XGS_LU,NOV-22_PLANNED_XGS_LU,DEC-22_PLANNED_XGS_LU
0,SD,SIOUX FALLS (MITCHELL) SD,"ABERDEEN, SD",6.0,ABRDSDCO,NaN,NaN,NaN,NaN,NaN
1,WA,SEATTLE-TACOMA WA,"ABERDEEN, WA",8.0,ABRDWA01,NaN,NaN,NaN,NaN,NaN
2,IA,DES MOINES-AMES IA,NON-CBSA,NaN,ACKLIACO,NaN,NaN,NaN,NaN,NaN
3,OR,EUGENE OR,"CORVALLIS, OR",5.0,ADAROR21,NaN,NaN,NaN,NaN,NaN
4,IA,DES MOINES-AMES IA,"DES MOINES-WEST DES MOINES, IA",14.0,ADELIACO,NaN,NaN,NaN,NaN,NaN


In [22]:
# join in actual build for XGS - manually update date column name

wc_ora_df = wc_ora_df.merge(luqualo_ora_xgs_df[['PRIMARY_WIRE_CENTER_ID','LU_QUAL_CNT']], how = 'left', on = 'PRIMARY_WIRE_CENTER_ID').rename(columns = {'LU_QUAL_CNT': 'OCT-22_ENABLED_XGS_LU'})
#luqualo_ora_gpon_df.head()

In [23]:
# join in actual build for GPON - manually update date column name
wc_ora_df = wc_ora_df.merge(luqualo_ora_gpon_df[['PRIMARY_WIRE_CENTER_ID','LU_QUAL_CNT']], how = 'left', on = 'PRIMARY_WIRE_CENTER_ID').rename(columns = {'LU_QUAL_CNT': 'OCT-22_ENABLED_GPON_LU'})


In [26]:
# check data
wc_ora_df[wc_ora_df['OCT-22_ENABLED_XGS_LU']>0]

,STATE,DMA,CBSA,OPPORTUNITY_SCORE,PRIMARY_WIRE_CENTER_ID,CORE_READINESS,SEP-22_PLANNED_XGS_LU,OCT-22_PLANNED_XGS_LU,NOV-22_PLANNED_XGS_LU,DEC-22_PLANNED_XGS_LU,OCT-22_ENABLED_XGS_LU,OCT-22_ENABLED_GPON_LU
682,UT,SALT LAKE CITY UT,"SALT LAKE CITY, UT",10.0,HLDYUTMA,XGS-PON - 180,NaN,72.0,NaN,1849.0,57.0,NaN


In [27]:
# clean data/reorder columns - manually adjust date column names
wc_ora_df = wc_ora_df[['PRIMARY_WIRE_CENTER_ID', 'STATE', 'DMA', 'CBSA', 'OPPORTUNITY_SCORE', 'CORE_READINESS', 'OCT-22_ENABLED_GPON_LU', 'OCT-22_ENABLED_XGS_LU', 'SEP-22_PLANNED_XGS_LU','OCT-22_PLANNED_XGS_LU','NOV-22_PLANNED_XGS_LU','DEC-22_PLANNED_XGS_LU']].rename(columns = {'Weighted Opportunity Score':'OPPORTUNITY_SCORE'})

In [28]:
# check data
wc_ora_df.head()

,PRIMARY_WIRE_CENTER_ID,STATE,DMA,CBSA,OPPORTUNITY_SCORE,CORE_READINESS,OCT-22_ENABLED_GPON_LU,OCT-22_ENABLED_XGS_LU,SEP-22_PLANNED_XGS_LU,OCT-22_PLANNED_XGS_LU,NOV-22_PLANNED_XGS_LU,DEC-22_PLANNED_XGS_LU
0,ABRDSDCO,SD,SIOUX FALLS (MITCHELL) SD,"ABERDEEN, SD",6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABRDWA01,WA,SEATTLE-TACOMA WA,"ABERDEEN, WA",8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ACKLIACO,IA,DES MOINES-AMES IA,NON-CBSA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ADAROR21,OR,EUGENE OR,"CORVALLIS, OR",5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ADELIACO,IA,DES MOINES-AMES IA,"DES MOINES-WEST DES MOINES, IA",14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
#Import Funding data
Construction_Funding_df = pd.read_excel(r'FundingDetailsExport/Details_Export_Table.xlsx',sheet_name = 'Export')

In [31]:
Construction_Funding_df['PHASE_ECD_LOGIC'] = Construction_Funding_df['PHASE_ECD_LOGIC'].dt.to_period('M')
Construction_Funding_df['PHASE_ECD_LOGIC'] = Construction_Funding_df['PHASE_ECD_LOGIC'].astype(str)
Construction_pivot = Construction_Funding_df.groupby(by = ["PRIMARY_WIRE_CENTER_ID","PHASE_ECD_LOGIC"], as_index = False).sum()
Construction_pivot = Construction_pivot.pivot(index='PRIMARY_WIRE_CENTER_ID', columns='PHASE_ECD_LOGIC')['PHASE_NUM_OF_LUS'].reset_index()
Construction_pivot = Construction_pivot.fillna(0)

In [32]:
# check this before manual date update below
Construction_pivot.head()

PHASE_ECD_LOGIC,PRIMARY_WIRE_CENTER_ID,2022-09,2022-10,2022-11,2022-12
0,ALSPFLXA,0.0,0.0,0.0,550.0
1,APPKFLXA,0.0,0.0,0.0,97.0
2,AUBNWA01,0.0,147.0,971.0,234.0
3,BLANMNBL,0.0,0.0,0.0,344.0
4,CLSPCOMA,2197.0,1167.0,0.0,0.0


In [33]:
# rename columns
Construction_Funding_df = Construction_pivot.copy()
Construction_Funding_df = Construction_Funding_df.groupby(by = ["PRIMARY_WIRE_CENTER_ID"], as_index = False).sum().rename(columns = {'2022-09': 'SEP-22_XGS_CONSTRUCTION_FUNDED',
                                                                                                           '2022-10': 'OCT-22_XGS_CONSTRUCTION_FUNDED',
                                                                                                           '2022-11': 'NOV-22_XGS_CONSTRUCTION_FUNDED',
                                                                                                           '2022-12': 'DEC-22_XGS_CONSTRUCTION_FUNDED'})

In [37]:
Construction_Funding_df.head()

PHASE_ECD_LOGIC,PRIMARY_WIRE_CENTER_ID,SEP-22_XGS_CONSTRUCTION_FUNDED,OCT-22_XGS_CONSTRUCTION_FUNDED,NOV-22_XGS_CONSTRUCTION_FUNDED,DEC-22_XGS_CONSTRUCTION_FUNDED
0,ALSPFLXA,0.0,0.0,0.0,550.0
1,APPKFLXA,0.0,0.0,0.0,97.0
2,AUBNWA01,0.0,147.0,971.0,234.0
3,BLANMNBL,0.0,0.0,0.0,344.0
4,CLSPCOMA,2197.0,1167.0,0.0,0.0


In [38]:
# join in construction funding to base table

wc_ora_df = wc_ora_df.merge(Construction_Funding_df, how = 'left', on = 'PRIMARY_WIRE_CENTER_ID')

In [39]:
# join in subscribers data sales_ora_df
sales_copper = sales_ora_df.loc[sales_ora_df['QUALTECH']=='COPPER',['WIRE_CENTER_ID','HH_CNT','EIS']].groupby(by = 'WIRE_CENTER_ID', as_index = False).sum()

In [40]:
sales_gpon = sales_ora_df.loc[sales_ora_df['QUALTECH']=='GPON',['WIRE_CENTER_ID','HH_CNT','EIS']].groupby(by = 'WIRE_CENTER_ID', as_index = False).sum()
sales_xgs = sales_ora_df.loc[sales_ora_df['QUALTECH']=='XGSPON',['WIRE_CENTER_ID','HH_CNT','EIS']].groupby(by = 'WIRE_CENTER_ID', as_index = False).sum()

In [41]:
# join in gpon sales - update date name
wc_ora_df = wc_ora_df.merge(sales_gpon, how = 'left', left_on = 'PRIMARY_WIRE_CENTER_ID', right_on = 'WIRE_CENTER_ID').drop('WIRE_CENTER_ID', axis = 1).rename(columns = {'HH_CNT':'SEP-22_GPON_ENABLED_HH',
                                                                                                                               'EIS':'SEP-22_GPON_ACTIVE_SUBS'})

In [42]:
# join in xgs sales - update date name
wc_ora_df = wc_ora_df.merge(sales_xgs, how = 'left', left_on = 'PRIMARY_WIRE_CENTER_ID', right_on = 'WIRE_CENTER_ID').drop('WIRE_CENTER_ID', axis = 1).rename(columns = {'HH_CNT':'SEP-22_XGS_ENABLED_HH',
                                                                                                                               'EIS':'SEP-22_XGS_ACTIVE_SUBS'})

In [43]:
# join in copper sales - update date name
wc_ora_df = wc_ora_df.merge(sales_copper, how = 'left', left_on = 'PRIMARY_WIRE_CENTER_ID', right_on = 'WIRE_CENTER_ID').drop('WIRE_CENTER_ID', axis = 1).rename(columns = {'HH_CNT':'SEP-22_COPPER_ENABLED_HH',
                                                                                                                               'EIS':'SEP-22_COPPER_ACTIVE_SUBS'})

In [44]:
# penetration rate by product calculation - update date related field names manually
wc_ora_df['SEP-22_COPPER_AGGREGATE_PENETRATION_RATE'] = wc_ora_df['SEP-22_COPPER_ACTIVE_SUBS'].fillna(0)/wc_ora_df['SEP-22_COPPER_ENABLED_HH'].fillna(0)
wc_ora_df['SEP-22_GPON_AGGREGATE_PENETRATION_RATE'] = wc_ora_df['SEP-22_GPON_ACTIVE_SUBS'].fillna(0)/wc_ora_df['SEP-22_GPON_ENABLED_HH'].fillna(0)
wc_ora_df['SEP-22_XGS_AGGREGATE_PENETRATION_RATE'] = wc_ora_df['SEP-22_XGS_ACTIVE_SUBS'].fillna(0)/wc_ora_df['SEP-22_XGS_ENABLED_HH'].fillna(0)

In [45]:
#fill na - update name columns manually
wc_ora_df[['OCT-22_ENABLED_GPON_LU','OCT-22_ENABLED_XGS_LU','OCT-22_PLANNED_XGS_LU','NOV-22_PLANNED_XGS_LU','DEC-22_PLANNED_XGS_LU','OCT-22_XGS_CONSTRUCTION_FUNDED','SEP-22_GPON_ENABLED_HH','SEP-22_GPON_ACTIVE_SUBS','SEP-22_XGS_ENABLED_HH','SEP-22_XGS_ACTIVE_SUBS','SEP-22_COPPER_ENABLED_HH','SEP-22_COPPER_ACTIVE_SUBS','SEP-22_COPPER_AGGREGATE_PENETRATION_RATE','SEP-22_GPON_AGGREGATE_PENETRATION_RATE','SEP-22_XGS_AGGREGATE_PENETRATION_RATE','NOV-22_XGS_CONSTRUCTION_FUNDED','DEC-22_XGS_CONSTRUCTION_FUNDED']] = wc_ora_df[['OCT-22_ENABLED_GPON_LU','OCT-22_ENABLED_XGS_LU','OCT-22_PLANNED_XGS_LU','NOV-22_PLANNED_XGS_LU','DEC-22_PLANNED_XGS_LU','OCT-22_XGS_CONSTRUCTION_FUNDED','SEP-22_GPON_ENABLED_HH','SEP-22_GPON_ACTIVE_SUBS','SEP-22_XGS_ENABLED_HH','SEP-22_XGS_ACTIVE_SUBS','SEP-22_COPPER_ENABLED_HH','SEP-22_COPPER_ACTIVE_SUBS','SEP-22_COPPER_AGGREGATE_PENETRATION_RATE','SEP-22_GPON_AGGREGATE_PENETRATION_RATE','SEP-22_XGS_AGGREGATE_PENETRATION_RATE','NOV-22_XGS_CONSTRUCTION_FUNDED','DEC-22_XGS_CONSTRUCTION_FUNDED']].fillna(0)

In [46]:
wc_ora_df[['OPPORTUNITY_SCORE','CORE_READINESS']] = wc_ora_df[['OPPORTUNITY_SCORE','CORE_READINESS']].fillna('N/A')

In [47]:
#final df ready for export
wc_ora_df.head()
#BNSNNCXA

,PRIMARY_WIRE_CENTER_ID,STATE,DMA,CBSA,OPPORTUNITY_SCORE,CORE_READINESS,OCT-22_ENABLED_GPON_LU,OCT-22_ENABLED_XGS_LU,SEP-22_PLANNED_XGS_LU,OCT-22_PLANNED_XGS_LU,NOV-22_PLANNED_XGS_LU,DEC-22_PLANNED_XGS_LU,SEP-22_XGS_CONSTRUCTION_FUNDED,OCT-22_XGS_CONSTRUCTION_FUNDED,NOV-22_XGS_CONSTRUCTION_FUNDED,DEC-22_XGS_CONSTRUCTION_FUNDED,SEP-22_GPON_ENABLED_HH,SEP-22_GPON_ACTIVE_SUBS,SEP-22_XGS_ENABLED_HH,SEP-22_XGS_ACTIVE_SUBS,SEP-22_COPPER_ENABLED_HH,SEP-22_COPPER_ACTIVE_SUBS,SEP-22_COPPER_AGGREGATE_PENETRATION_RATE,SEP-22_GPON_AGGREGATE_PENETRATION_RATE,SEP-22_XGS_AGGREGATE_PENETRATION_RATE
0,ABRDSDCO,SD,SIOUX FALLS (MITCHELL) SD,"ABERDEEN, SD",6.0,N/A,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14578.0,732.0,0.050213,0.000000,0.0
1,ABRDWA01,WA,SEATTLE-TACOMA WA,"ABERDEEN, WA",8.0,N/A,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,7970.0,2320.0,0.0,0.0,4621.0,904.0,0.195629,0.291092,0.0
2,ACKLIACO,IA,DES MOINES-AMES IA,NON-CBSA,N/A,N/A,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1119.0,316.0,0.282395,0.000000,0.0
3,ADAROR21,OR,EUGENE OR,"CORVALLIS, OR",5.0,N/A,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1239.0,410.0,0.330912,0.000000,0.0
4,ADELIACO,IA,DES MOINES-AMES IA,"DES MOINES-WEST DES MOINES, IA",14.0,N/A,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,820.0,1242.0,0.0,0.0,2528.0,1286.0,0.508703,1.514634,0.0


In [48]:
import time

# export
TodaysDate = time.strftime("%m%d%y")
excelfilename = "Output/" + TodaysDate +"_WC_Universe_Cumulative_Rpt_Data.xlsx"

with pd.ExcelWriter(excelfilename) as writer:  
    wc_ora_df.to_excel(writer, sheet_name='Cumulative Data', index = False)
    luqualo_ora_gpon_df.to_excel(writer, sheet_name='LUQUAL Build Details GPON', index = False)
    luqualo_ora_xgs_df.to_excel(writer, sheet_name='LUQUAL Build Details XGS', index = False)
    Construction_Funding_df.to_excel(writer, sheet_name='ST Construction Funding Details', index = False)
    sales_ora_df.to_excel(writer, sheet_name='Sales Data Details', index = False)